In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
from sklearn import preprocessing
from sklearn import datasets

In [3]:
X = pd.read_csv("D:\\Northeastern\\50 pct undersample final data\\X_val.csv")#pd.read_csv("D:\\Northeastern\\50 pct undersample final data\\X_train.csv")

# Gender
We treat 'unknown' as a 3rd category of gender with the assumption being that the user's decision to not provide gender may be relevant to our predictions

In [4]:
X.loc[X["gender"] == "male","gender"] = "M"
X.loc[X["gender"] == "female","gender"] = "F"
X.loc[:,"gender"]=X["gender"].fillna("U")

enc_gender = preprocessing.OneHotEncoder(drop="first",categories="auto")
gender_onehot = enc_gender.fit_transform(X.gender.values.reshape(-1,1)).toarray()
# Drop old gender column and merge new onehot encoded columns
X=X.drop(["gender"],axis=1)
X["gender_male"] = gender_onehot[:,0]
X["gender_unknown"] = gender_onehot[:,1]
X.head()

,msno,city,bd,registered_via,registration_init_time,gender_male,gender_unknown
0,30OBk7llwUOPuZepCHXi4o++UtD1+rWJweVblworcrM=,13,36,9,20101128,0.0,0.0
1,jnUsnKljcVx/uSBXChZku9GPqzoB/CgIoZMm0j0Uoko=,1,0,7,20131211,0.0,1.0
2,qnd7B4B0eWqUmKpvoOY9ycpSvncMfih183DwlujR/0g=,22,19,9,20120120,1.0,0.0
3,D7OvyeuqeYTav6A8Lt7cperdnjW5VO7ZPCiIg1e2wmI=,1,0,7,20130705,0.0,1.0
4,GbafNG8rtEg+kWmwV++K/vit9GvEpl719/A8uWwRpb4=,6,23,9,20141018,1.0,0.0


# Registration Date
We convert registration date into an integer containing the number of days since registration. Those values are then standardized.

In [5]:
reg_date = pd.to_datetime(X.registration_init_time, format="%Y%m%d")
# Assuming here that our training date for predicting February 2017 is Jan 1, 2017
train_date = pd.to_datetime("20170131", format="%Y%m%d")
reg_days = (train_date - reg_date).dt.days

reg_days_scaled = preprocessing.scale(reg_days)
X["reg_days_scaled"] = pd.Series(reg_days_scaled)
X=X.drop(["registration_init_time"], axis=1)
X.head()

,msno,city,bd,registered_via,gender_male,gender_unknown,reg_days_scaled
0,30OBk7llwUOPuZepCHXi4o++UtD1+rWJweVblworcrM=,13,36,9,0.0,0.0,0.597195
1,jnUsnKljcVx/uSBXChZku9GPqzoB/CgIoZMm0j0Uoko=,1,0,7,0.0,1.0,-0.441313
2,qnd7B4B0eWqUmKpvoOY9ycpSvncMfih183DwlujR/0g=,22,19,9,1.0,0.0,0.205765
3,D7OvyeuqeYTav6A8Lt7cperdnjW5VO7ZPCiIg1e2wmI=,1,0,7,0.0,1.0,-0.292420
4,GbafNG8rtEg+kWmwV++K/vit9GvEpl719/A8uWwRpb4=,6,23,9,1.0,0.0,-0.732545


# Registration Method
We one-hot encode the registration methods

In [6]:
enc_registration = preprocessing.OneHotEncoder(drop="first",categories="auto")
registration_onehot = enc_registration.fit_transform(X.registered_via.values.reshape(-1,1)).toarray()
registration_df = pd.DataFrame(registration_onehot)
registration_df.columns = enc_registration.get_feature_names(["registered_via"])
X = pd.concat([X,registration_df], axis=1)
X = X.drop(["registered_via"],axis=1)
X.head()

,msno,city,bd,gender_male,gender_unknown,reg_days_scaled,registered_via_4,registered_via_7,registered_via_9
0,30OBk7llwUOPuZepCHXi4o++UtD1+rWJweVblworcrM=,13,36,0.0,0.0,0.597195,0.0,0.0,1.0
1,jnUsnKljcVx/uSBXChZku9GPqzoB/CgIoZMm0j0Uoko=,1,0,0.0,1.0,-0.441313,0.0,1.0,0.0
2,qnd7B4B0eWqUmKpvoOY9ycpSvncMfih183DwlujR/0g=,22,19,1.0,0.0,0.205765,0.0,0.0,1.0
3,D7OvyeuqeYTav6A8Lt7cperdnjW5VO7ZPCiIg1e2wmI=,1,0,0.0,1.0,-0.292420,0.0,1.0,0.0
4,GbafNG8rtEg+kWmwV++K/vit9GvEpl719/A8uWwRpb4=,6,23,1.0,0.0,-0.732545,0.0,0.0,1.0


# City
We one-hot encode the city

In [7]:
enc_city = preprocessing.OneHotEncoder(drop="first",categories="auto")
city_onehot = enc_city.fit_transform(X.city.values.reshape(-1,1)).toarray()
city_df = pd.DataFrame(city_onehot)
city_df.columns = enc_city.get_feature_names(["city"])
X = pd.concat([X,city_df], axis=1)
X = X.drop(["city"],axis=1)
X.head()

,msno,bd,gender_male,gender_unknown,reg_days_scaled,registered_via_4,registered_via_7,registered_via_9,city_3,city_4,...,city_13,city_14,city_15,city_16,city_17,city_18,city_19,city_20,city_21,city_22
0,30OBk7llwUOPuZepCHXi4o++UtD1+rWJweVblworcrM=,36,0.0,0.0,0.597195,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,jnUsnKljcVx/uSBXChZku9GPqzoB/CgIoZMm0j0Uoko=,0,0.0,1.0,-0.441313,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,qnd7B4B0eWqUmKpvoOY9ycpSvncMfih183DwlujR/0g=,19,1.0,0.0,0.205765,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,D7OvyeuqeYTav6A8Lt7cperdnjW5VO7ZPCiIg1e2wmI=,0,0.0,1.0,-0.292420,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,GbafNG8rtEg+kWmwV++K/vit9GvEpl719/A8uWwRpb4=,23,1.0,0.0,-0.732545,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Birthday

Scaling the BD values. Others have indicated that this feature is predictive (despite the strange, negative values)

In [8]:
X["bd"] = preprocessing.scale(X.bd)
X.head()

,msno,bd,gender_male,gender_unknown,reg_days_scaled,registered_via_4,registered_via_7,registered_via_9,city_3,city_4,...,city_13,city_14,city_15,city_16,city_17,city_18,city_19,city_20,city_21,city_22
0,30OBk7llwUOPuZepCHXi4o++UtD1+rWJweVblworcrM=,0.817102,0.0,0.0,0.597195,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,jnUsnKljcVx/uSBXChZku9GPqzoB/CgIoZMm0j0Uoko=,-0.711241,0.0,1.0,-0.441313,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,qnd7B4B0eWqUmKpvoOY9ycpSvncMfih183DwlujR/0g=,0.095385,1.0,0.0,0.205765,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,D7OvyeuqeYTav6A8Lt7cperdnjW5VO7ZPCiIg1e2wmI=,-0.711241,0.0,1.0,-0.292420,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,GbafNG8rtEg+kWmwV++K/vit9GvEpl719/A8uWwRpb4=,0.265201,1.0,0.0,-0.732545,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
X.to_csv("D:\\Northeastern\\50 pct undersample final data\\X_val_transformed.csv",index=False)#to_csv("D:\\Northeastern\\50 pct undersample final data\\X_train_transformed.csv",index=False)